In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_csv("Train_Test_IoT_GPS_Tracker.csv")

In [3]:
df

,ts,date,time,latitude,longitude,label,type
0,1556242299,25-Apr-19,18:31:39,116.521704,132.162504,1,ddos
1,1556242301,25-Apr-19,18:31:41,121.742933,135.072500,1,ddos
2,1556242306,25-Apr-19,18:31:46,121.795929,135.110198,1,ddos
3,1556242309,25-Apr-19,18:31:49,116.611216,132.224555,1,ddos
4,1556242311,25-Apr-19,18:31:51,121.841526,135.151362,1,ddos
...,...,...,...,...,...,...,...
58955,1554202928,2-Apr-19,4:02:08,1.028628,10.810522,0,normal
58956,1554202928,2-Apr-19,4:02:08,0.345361,10.727989,0,normal
58957,1554202928,2-Apr-19,4:02:08,0.423384,10.508036,0,normal
58958,1554202928,2-Apr-19,4:02:08,0.445292,10.381356,0,normal


In [4]:
encodingLabels=pd.get_dummies(df.type)
df=pd.concat([df, encodingLabels], axis=1)

In [5]:
df

,ts,date,time,latitude,longitude,label,type,backdoor,ddos,injection,normal,password,ransomware,scanning,xss
0,1556242299,25-Apr-19,18:31:39,116.521704,132.162504,1,ddos,0,1,0,0,0,0,0,0
1,1556242301,25-Apr-19,18:31:41,121.742933,135.072500,1,ddos,0,1,0,0,0,0,0,0
2,1556242306,25-Apr-19,18:31:46,121.795929,135.110198,1,ddos,0,1,0,0,0,0,0,0
3,1556242309,25-Apr-19,18:31:49,116.611216,132.224555,1,ddos,0,1,0,0,0,0,0,0
4,1556242311,25-Apr-19,18:31:51,121.841526,135.151362,1,ddos,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58955,1554202928,2-Apr-19,4:02:08,1.028628,10.810522,0,normal,0,0,0,1,0,0,0,0
58956,1554202928,2-Apr-19,4:02:08,0.345361,10.727989,0,normal,0,0,0,1,0,0,0,0
58957,1554202928,2-Apr-19,4:02:08,0.423384,10.508036,0,normal,0,0,0,1,0,0,0,0
58958,1554202928,2-Apr-19,4:02:08,0.445292,10.381356,0,normal,0,0,0,1,0,0,0,0


In [6]:
X=df[["longitude","latitude"]]
y=df[['normal','backdoor','ddos', 'password',  'injection', 'ransomware',  'xss' ,'scanning']]

In [7]:
from sklearn.preprocessing import StandardScaler
 

 
# Initialise the Scaler
scaler = StandardScaler()
 
# To scale data
X=scaler.fit_transform(X)

X

array([[ 1.0038727 ,  0.93420817],
       [ 1.04265624,  1.00407152],
       [ 1.04315867,  1.00478064],
       ...,
       [-0.61750105, -0.61926088],
       [-0.61918939, -0.61896775],
       [-0.61966083, -0.6177983 ]])

In [8]:
y

,normal,backdoor,ddos,password,injection,ransomware,xss,scanning
0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
58955,1,0,0,0,0,0,0,0
58956,1,0,0,0,0,0,0,0
58957,1,0,0,0,0,0,0,0
58958,1,0,0,0,0,0,0,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

print("Train Data size:", len(X_train))
print("Test Data size", len(X_test))

Train Data size: 44220
Test Data size 14740


In [10]:
X_train.shape,y_train.shape

((44220, 2), (44220, 8))

In [12]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(2, 1)))
cnn.add(Convolution1D(64, 3, padding="same", activation="relu"))
cnn.add(MaxPooling1D(pool_size=(2),padding="same"))
cnn.add(Convolution1D(128, 3, padding="same", activation="relu"))
cnn.add(Convolution1D(128, 3, padding="same", activation="relu"))
cnn.add(MaxPooling1D(pool_size=(2),padding="same"))
cnn.add(Flatten())
cnn.add(Dropout(0.1))
cnn.add(Dense(8, activation="softmax"))

In [13]:
cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [14]:
checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('cnntrainanalysisGPSCNNwithScaling.csv',separator=',', append=False)

In [15]:
import time
start = time.time()

history=cnn.fit(X_train, y_train,batch_size=256,epochs=500,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("cnn_modelGPSwithScalingnoLSTM.hdf5")

end = time.time()
diff = (end - start)/60
print('Training time (in Minutes):', diff)

Epoch 1/500
173/173 [==============================] - 2s 9ms/step - loss: 1.1260 - accuracy: 0.6041 - val_loss: 0.8564 - val_accuracy: 0.6505
Epoch 2/500
173/173 [==============================] - 1s 8ms/step - loss: 0.8332 - accuracy: 0.6446 - val_loss: 0.7872 - val_accuracy: 0.6548
Epoch 3/500
173/173 [==============================] - 1s 8ms/step - loss: 0.8009 - accuracy: 0.6480 - val_loss: 0.7725 - val_accuracy: 0.6588
Epoch 4/500
173/173 [==============================] - 1s 8ms/step - loss: 0.7876 - accuracy: 0.6553 - val_loss: 0.7551 - val_accuracy: 0.6828
Epoch 5/500
173/173 [==============================] - 1s 8ms/step - loss: 0.7633 - accuracy: 0.6799 - val_loss: 0.7269 - val_accuracy: 0.7018
Epoch 6/500
173/173 [==============================] - 1s 8ms/step - loss: 0.7346 - accuracy: 0.6910 - val_loss: 0.6938 - val_accuracy: 0.7130
Epoch 7/500
173/173 [==============================] - 1s 8ms/step - loss: 0.7044 - accuracy: 0.6979 - val_loss: 0.6767 - val_accuracy: 0.7231

Epoch 28/500
173/173 [==============================] - 2s 10ms/step - loss: 0.5678 - accuracy: 0.7688 - val_loss: 0.5285 - val_accuracy: 0.7908
Epoch 29/500
173/173 [==============================] - 3s 15ms/step - loss: 0.5604 - accuracy: 0.7733 - val_loss: 0.5376 - val_accuracy: 0.7777
Epoch 30/500
173/173 [==============================] - 3s 14ms/step - loss: 0.5529 - accuracy: 0.7766 - val_loss: 0.5165 - val_accuracy: 0.7943
Epoch 31/500
173/173 [==============================] - 2s 14ms/step - loss: 0.5450 - accuracy: 0.7800 - val_loss: 0.5479 - val_accuracy: 0.7665
Epoch 32/500
173/173 [==============================] - 2s 12ms/step - loss: 0.5473 - accuracy: 0.7804 - val_loss: 0.5383 - val_accuracy: 0.7869
Epoch 33/500
173/173 [==============================] - 2s 12ms/step - loss: 0.5405 - accuracy: 0.7821 - val_loss: 0.4993 - val_accuracy: 0.7985
Epoch 34/500
173/173 [==============================] - 2s 10ms/step - loss: 0.5304 - accuracy: 0.7863 - val_loss: 0.4959 - val_ac

Epoch 55/500
173/173 [==============================] - 2s 12ms/step - loss: 0.4867 - accuracy: 0.8028 - val_loss: 0.4552 - val_accuracy: 0.8166
Epoch 56/500
173/173 [==============================] - 2s 11ms/step - loss: 0.4736 - accuracy: 0.8076 - val_loss: 0.4498 - val_accuracy: 0.8136
Epoch 57/500
173/173 [==============================] - 2s 10ms/step - loss: 0.4728 - accuracy: 0.8087 - val_loss: 0.4342 - val_accuracy: 0.8220
Epoch 58/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4746 - accuracy: 0.8064 - val_loss: 0.4312 - val_accuracy: 0.8251
Epoch 59/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4693 - accuracy: 0.8097 - val_loss: 0.5089 - val_accuracy: 0.7832
Epoch 60/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4697 - accuracy: 0.8089 - val_loss: 0.4315 - val_accuracy: 0.8276
Epoch 61/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4706 - accuracy: 0.8072 - val_loss: 0.4605 - val_accura

Epoch 82/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4341 - accuracy: 0.8235 - val_loss: 0.4020 - val_accuracy: 0.8367
Epoch 83/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4335 - accuracy: 0.8231 - val_loss: 0.4078 - val_accuracy: 0.8288
Epoch 84/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4350 - accuracy: 0.8230 - val_loss: 0.3960 - val_accuracy: 0.8402
Epoch 85/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4345 - accuracy: 0.8245 - val_loss: 0.4154 - val_accuracy: 0.8315
Epoch 86/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4370 - accuracy: 0.8220 - val_loss: 0.4260 - val_accuracy: 0.8220
Epoch 87/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4319 - accuracy: 0.8246 - val_loss: 0.4031 - val_accuracy: 0.8332
Epoch 88/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4302 - accuracy: 0.8262 - val_loss: 0.4127 - val_accuracy:

Epoch 109/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4161 - accuracy: 0.8336 - val_loss: 0.3946 - val_accuracy: 0.8372
Epoch 110/500
173/173 [==============================] - 2s 10ms/step - loss: 0.4189 - accuracy: 0.8325 - val_loss: 0.4247 - val_accuracy: 0.8210
Epoch 111/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4093 - accuracy: 0.8350 - val_loss: 0.3717 - val_accuracy: 0.8533
Epoch 112/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4092 - accuracy: 0.8372 - val_loss: 0.3780 - val_accuracy: 0.8468
Epoch 113/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4071 - accuracy: 0.8356 - val_loss: 0.3705 - val_accuracy: 0.8497
Epoch 114/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4070 - accuracy: 0.8377 - val_loss: 0.3758 - val_accuracy: 0.8460
Epoch 115/500
173/173 [==============================] - 2s 9ms/step - loss: 0.4111 - accuracy: 0.8353 - val_loss: 0.3778 - val_a

...

173/173 [==============================] - 1s 8ms/step - loss: 0.3289 - accuracy: 0.8699 - val_loss: 0.2999 - val_accuracy: 0.8801
Epoch 376/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3294 - accuracy: 0.8698 - val_loss: 0.3132 - val_accuracy: 0.8728
Epoch 377/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3238 - accuracy: 0.8735 - val_loss: 0.2988 - val_accuracy: 0.8811
Epoch 378/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3200 - accuracy: 0.8736 - val_loss: 0.3255 - val_accuracy: 0.8655
Epoch 379/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3215 - accuracy: 0.8715 - val_loss: 0.3060 - val_accuracy: 0.8793
Epoch 380/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3246 - accuracy: 0.8713 - val_loss: 0.3052 - val_accuracy: 0.8773
Epoch 381/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3260 - accuracy: 0.8716 - val_loss: 0.3005 - val_accuracy: 0.8792

Epoch 402/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3218 - accuracy: 0.8724 - val_loss: 0.3022 - val_accuracy: 0.8806
Epoch 403/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3158 - accuracy: 0.8759 - val_loss: 0.3116 - val_accuracy: 0.8748
Epoch 404/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3201 - accuracy: 0.8735 - val_loss: 0.3048 - val_accuracy: 0.8780
Epoch 405/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3202 - accuracy: 0.8739 - val_loss: 0.2987 - val_accuracy: 0.8808
Epoch 406/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3219 - accuracy: 0.8740 - val_loss: 0.3062 - val_accuracy: 0.8777
Epoch 407/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3211 - accuracy: 0.8714 - val_loss: 0.3142 - val_accuracy: 0.8748
Epoch 408/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3185 - accuracy: 0.8739 - val_loss: 0.3662 - val_ac

Epoch 429/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3198 - accuracy: 0.8739 - val_loss: 0.2988 - val_accuracy: 0.8789
Epoch 430/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3158 - accuracy: 0.8752 - val_loss: 0.3141 - val_accuracy: 0.8731
Epoch 431/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3116 - accuracy: 0.8771 - val_loss: 0.2973 - val_accuracy: 0.8792
Epoch 432/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3135 - accuracy: 0.8766 - val_loss: 0.3051 - val_accuracy: 0.8754
Epoch 433/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3133 - accuracy: 0.8769 - val_loss: 0.2921 - val_accuracy: 0.8794
Epoch 434/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3179 - accuracy: 0.8753 - val_loss: 0.3023 - val_accuracy: 0.8772
Epoch 435/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3102 - accuracy: 0.8763 - val_loss: 0.3001 - val_ac

Epoch 456/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3158 - accuracy: 0.8758 - val_loss: 0.3147 - val_accuracy: 0.8681
Epoch 457/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3112 - accuracy: 0.8776 - val_loss: 0.3124 - val_accuracy: 0.8729
Epoch 458/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3132 - accuracy: 0.8760 - val_loss: 0.3014 - val_accuracy: 0.8788
Epoch 459/500
173/173 [==============================] - 2s 9ms/step - loss: 0.3109 - accuracy: 0.8768 - val_loss: 0.2963 - val_accuracy: 0.8793
Epoch 460/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3152 - accuracy: 0.8755 - val_loss: 0.3296 - val_accuracy: 0.8659
Epoch 461/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3134 - accuracy: 0.8744 - val_loss: 0.2938 - val_accuracy: 0.8828
Epoch 462/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3151 - accuracy: 0.8757 - val_loss: 0.3120 - val_ac

Epoch 483/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3090 - accuracy: 0.8784 - val_loss: 0.2985 - val_accuracy: 0.8792
Epoch 484/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3170 - accuracy: 0.8744 - val_loss: 0.2980 - val_accuracy: 0.8788
Epoch 485/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3136 - accuracy: 0.8753 - val_loss: 0.3343 - val_accuracy: 0.8672
Epoch 486/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3101 - accuracy: 0.8763 - val_loss: 0.2884 - val_accuracy: 0.8874
Epoch 487/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3134 - accuracy: 0.8769 - val_loss: 0.3051 - val_accuracy: 0.8783
Epoch 488/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3086 - accuracy: 0.8772 - val_loss: 0.3069 - val_accuracy: 0.8748
Epoch 489/500
173/173 [==============================] - 1s 8ms/step - loss: 0.3138 - accuracy: 0.8748 - val_loss: 0.2989 - val_ac

In [16]:
dfanalysis=pd.read_csv("cnntrainanalysisGPSCNNwithScaling.csv")
dfanalysis.describe()

,epoch,accuracy,loss,val_accuracy,val_loss
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,249.500000,0.846523,0.382246,0.851828,0.364194
std,144.481833,0.039409,0.091522,0.036674,0.084481
min,0.000000,0.604071,0.304739,0.650543,0.288275
25%,124.750000,0.838275,0.326116,0.847795,0.312415
50%,249.500000,0.860029,0.351204,0.863704,0.334898
75%,374.250000,0.870969,0.401022,0.873406,0.377843
max,499.000000,0.880032,1.125991,0.887381,0.856425


In [17]:
from keras.models import load_model
model = load_model('cnn_modelGPSwithScalingnoLSTM.hdf5')

In [18]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

461/461 [==============================] - 1s 2ms/step - loss: 0.2999 - accuracy: 0.8780

Loss: 0.30, Accuracy: 87.80%


In [19]:
y_pred=model.predict(X_test) 
#y_pred=np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)
#y_pred=model.predict(X_test) 
#y_pred=np.argmax(y_pred,axis=1)
#y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average=None)
precision = precision_score(y_test, y_pred , average=None)
f1 = f1_score(y_test, y_pred, average=None)

In [20]:
recallList=recall.tolist()
precisionList=precision.tolist()
f1lList=f1.tolist()

In [21]:
name_class=['normal','backdoor','ddos', 'password',  'injection', 'ransomware',  'xss' ,'scanning']
name_class.sort()
name_class

['backdoor',
 'ddos',
 'injection',
 'normal',
 'password',
 'ransomware',
 'scanning',
 'xss']

In [22]:
recallDict={i:j for i,j in zip(name_class,recallList)}
precisionDict={i:j for i,j in zip(name_class,precisionList)}
f1Dict={i:j for i,j in zip(name_class,f1lList)}

## Average Recall

In [23]:
recallDict

{'backdoor': 0.9718422252621979,
 'ddos': 0.6867469879518072,
 'injection': 0.7243027888446215,
 'normal': 0.5144270403957131,
 'password': 0.9011764705882352,
 'ransomware': 0.5398860398860399,
 'scanning': 0.16911764705882354,
 'xss': 0.8873239436619719}

## Average Precision

In [24]:
precisionDict

{'backdoor': 0.9627329192546584,
 'ddos': 0.8887733887733887,
 'injection': 0.8624288425047438,
 'normal': 0.7473053892215569,
 'password': 0.747074122236671,
 'ransomware': 0.8168103448275862,
 'scanning': 0.6388888888888888,
 'xss': 0.984375}

## Average F1 score

In [25]:
f1Dict

{'backdoor': 0.9672661258296931,
 'ddos': 0.7748074309016764,
 'injection': 0.7873538328280641,
 'normal': 0.6093749999999999,
 'password': 0.8169214361891219,
 'ransomware': 0.6500857632933104,
 'scanning': 0.2674418604651163,
 'xss': 0.9333333333333333}

In [26]:
test_loss, test_acc = cnn.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

461/461 [==============================] - 1s 2ms/step - loss: 0.2999 - accuracy: 0.8780
Test accuracy 0.8780189752578735
Test loss 0.2999245822429657


In [27]:
train_loss, train_acc = cnn.evaluate(X_train, y_train)

print("Train accuracy", train_acc)
print("Train loss", train_loss)

1382/1382 [==============================] - 2s 2ms/step - loss: 0.2913 - accuracy: 0.8848
Train accuracy 0.8847806453704834
Train loss 0.29130980372428894
